# Cleaning up and Creating Content Vocabulary

### Import Everything
Just easier that way

In [107]:
import numpy as np
import pandas as pd
import os

# Word processing libraries
import re
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

#Nan
from cmath import nan

In [108]:
vocab = pd.read_excel('DATA/Protest is SA   SAS rules V2 - Separated.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'DATA/Protest is SA   SAS rules V2 - Separated.xlsx'

In [ ]:
vocab.shape

(100, 6)

In [ ]:
vocab.head(3)

,Label,Concept,Phrases,Conditional_Phrases,Afrikaans_Phrases,Rule
0,Grievance,Capitalism,"Capitalism,neoliberalism,privatisation,privati...",NaN,"Kapitalisme, privatisasie, privatiseer, neolib...",Multiple grievances ok
1,Grievance,Values,"Abuse,against xenophobia",NaN,"mishandeling, teen xenofobie",Multiple grievances ok
2,Grievance,Conditions,"poverty,unemployment,jobs,job",NaN,"armoede, werkloosheid, werke",Multiple grievances ok


In [ ]:
vocab = vocab.dropna(how = 'all')

In [ ]:
vocab.shape

(92, 6)

In [ ]:
vocab[(vocab['Afrikaans_Phrases'].isnull())]

,Label,Concept,Phrases,Conditional_Phrases,Afrikaans_Phrases,Rule
9,Grievance,Labour related,"Work,workers,Company,Employ,employer,employee,...","corrupt,fired,dismissed,policy,other people,ho...",NaN,Multiple grievances ok


### Drop Rule
Important to take note of what the rule column means: </br>
Rule indicates whether text can be grouped into multiple categories </br>
Rules are as follows:
Can Contain Multiple:
* Grievance
* Actors
* Locations
* Weapons
* Eventualities
* Curiosity

Can Only Have 1:
* Trigger
* Tactic</br>
Re-think this because it could be possible to have multiple. </br>
Events could be a combination of multiple things </br>
eg: could be a gathering and a march</br>

Discard:
* Non-protest</br>
    Unless it contains:
    * Grievance
    * Trigger
    * Tactic
    * Actor -> only for "16 days of activism"


Now we can drop Rule

In [ ]:
vocab = vocab.drop('Rule', axis = 1)

In [ ]:
vocab.head(3)

,Label,Concept,Phrases,Conditional_Phrases,Afrikaans_Phrases
0,Grievance,Capitalism,"Capitalism,neoliberalism,privatisation,privati...",NaN,"Kapitalisme, privatisasie, privatiseer, neolib..."
1,Grievance,Values,"Abuse,against xenophobia",NaN,"mishandeling, teen xenofobie"
2,Grievance,Conditions,"poverty,unemployment,jobs,job",NaN,"armoede, werkloosheid, werke"


### Drop Afrikaans for now
We can always create functionality for it at a later date

In [ ]:
vocab = vocab.drop('Afrikaans_Phrases', axis = 1)

In [ ]:
vocab.head(3)

,Label,Concept,Phrases,Conditional_Phrases
0,Grievance,Capitalism,"Capitalism,neoliberalism,privatisation,privati...",NaN
1,Grievance,Values,"Abuse,against xenophobia",NaN
2,Grievance,Conditions,"poverty,unemployment,jobs,job",NaN


### Lemmatise Vocabulary so that it can easily be matched to text in tweet

In [ ]:
# Define the function to implement POS tagging:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


# Define the main function to clean text in various ways:
def clean_text(text):
    if text != text:
        return nan

    else:
        # Apply regex expressions first before converting string to list of tokens/words:

        # 5. Convert text to lowercase
        text = text.lower()
        
        # 6. tokenize text and remove punctuation
        text = [word.strip(string.punctuation) for word in text.split(" ")]
        
        # 7. remove numbers
        text = [word for word in text if not any(c.isdigit() for c in word)]
        
        # 8. remove stop words
        stop = stopwords.words('english')
        text = [x for x in text if x not in stop]
        
        # 9. remove empty tokens
        text = [t for t in text if len(t) > 0]
        
        # 10. pos tag text and lemmatize text
        pos_tags = pos_tag(text)
        text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
        
        # 11. remove words with only one letter
        text = [t for t in text if len(t) > 1]
        
        # join all
        text = " ".join(text)
    
    return(text)

In [ ]:
vocab.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 0 to 99
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Label                92 non-null     object
 1   Concept              92 non-null     object
 2   Phrases              92 non-null     object
 3   Conditional_Phrases  15 non-null     object
dtypes: object(4)
memory usage: 3.6+ KB


In [ ]:
vocab['cleaned_phrases'] = vocab['Phrases'].apply(lambda x: clean_text(x))
vocab['cleaned_conditional_phrases'] = vocab['Conditional_Phrases'].apply(lambda x: clean_text(x))

In [ ]:
vocab.Phrases = vocab.Phrases.str. split(',')
vocab.Conditional_Phrases = vocab.Conditional_Phrases.str.split(',')
vocab.cleaned_phrases = vocab.cleaned_phrases.str. split(' ')
vocab.cleaned_conditional_phrases = vocab.cleaned_conditional_phrases.str. split(' ')

### Matching Content Analysis
Do content analysis for both lematised and non lematised text to make sure

In [ ]:
vocab.head(10)

,Label,Concept,Phrases,Conditional_Phrases,cleaned_phrases,cleaned_conditional_phrases
0,Grievance,Capitalism,"[Capitalism, neoliberalism, privatisation, pri...",NaN,"[capitalism,neoliberalism,privatisation,privat...",NaN
1,Grievance,Values,"[Abuse, against xenophobia]",NaN,"[abuse,against, xenophobia]",NaN
2,Grievance,Conditions,"[poverty, unemployment, jobs, job]",NaN,"[poverty,unemployment,jobs,job]",NaN
3,Grievance,Contract end,"[contracts expire, project end, completed, con...",NaN,"[contract, expire,project, end,completed,contr...",NaN
4,Grievance,Community recognition,"[their rights, recognition]",NaN,"[rights,recognition]",NaN
5,Grievance,Demolitions,"[destroyed, evicted, demolished, relocated, re...",NaN,"[destroyed,evicted,demolished,relocated,remova...",NaN
6,Grievance,Education,"[School, university, students, student, varsit...","[fees, costs, teacher, permission, results, po...","[school,university,students,student,varsity,va...","[fees,costs,teacher,permission,results,policy,..."
7,Grievance,Election outcome,[Election],"[outcome, result, winner, unfair, cheat, wrong...",[election],"[outcome,result,winner,unfair,cheat,wrong,coun..."
8,Grievance,Electricity,"[Electricity, Power, connections, loadshedding...","[cost, price, supply, cut, disconnect, loadshe...","[electricity,power,connections,loadshedding,lo...","[cost,price,supply,cut,disconnect,loadshedding..."
9,Grievance,Labour related,"[Work, workers, Company, Employ, employer, emp...","[corrupt, fired, dismissed, policy, other peop...","[work,workers,company,employ,employer,employee...","[corrupt,fired,dismissed,policy,other, people,..."


In [ ]:
print(vocab.Phrases[0])

['Capitalism', 'neoliberalism', 'privatisation', 'privatise', 'privatize', 'privatization']


### Split Up various Labels Into Own Dataframe

#### Grievances

In [ ]:
grievances = vocab.loc[vocab['Label'] == 'Grievance']
grievances = grievances.drop('Label', axis = 1)
grievances.to_excel('DATA/Vocabularies/grievances.xlsx')
grievances.shape

(36, 5)

In [ ]:
grievances.head()

,Concept,Phrases,Conditional_Phrases,cleaned_phrases,cleaned_conditional_phrases
0,Capitalism,"[Capitalism, neoliberalism, privatisation, pri...",NaN,"[capitalism,neoliberalism,privatisation,privat...",NaN
1,Values,"[Abuse, against xenophobia]",NaN,"[abuse,against, xenophobia]",NaN
2,Conditions,"[poverty, unemployment, jobs, job]",NaN,"[poverty,unemployment,jobs,job]",NaN
3,Contract end,"[contracts expire, project end, completed, con...",NaN,"[contract, expire,project, end,completed,contr...",NaN
4,Community recognition,"[their rights, recognition]",NaN,"[rights,recognition]",NaN


#### Trigger

In [ ]:
trigger = vocab.loc[vocab['Label'] == 'Trigger']
trigger = trigger.drop('Label', axis = 1)
trigger.to_excel('DATA/Vocabularies/trigger.xlsx')
trigger.shape

(13, 5)

In [ ]:
trigger.head()

,Concept,Phrases,Conditional_Phrases,cleaned_phrases,cleaned_conditional_phrases
37,Court hearing,"[court, hearing]",NaN,"[court,hearing]",NaN
38,Project begin,"[not local, not from area, outside, only be lo...",NaN,"[local,not, area,outside,only, local,only, loc...",NaN
39,National Strike,"[COSATU, NUMSA, national strike]",NaN,"[cosatu,numsa,national, strike]",NaN
40,Dismissals,"[Fired, suspended, dismissed, discipline, diss...",NaN,"[fired,suspended,dismissed,discipline,dissmiss...",NaN
41,Wage disputes,"[Payment, salary, cheque, overtime, wage, wage...",NaN,"[payment,salary,cheque,overtime,wage,wages,rem...",NaN


#### Tactic

In [ ]:
tactic = vocab.loc[vocab['Label'] == 'Tactic']
tactic = tactic.drop('Label', axis = 1)
tactic.to_excel('DATA/Vocabularies/tactic.xlsx')
tactic.shape

(14, 5)

In [ ]:
tactic.head()

,Concept,Phrases,Conditional_Phrases,cleaned_phrases,cleaned_conditional_phrases
51,Hostage,"[Captive, hostage, lock-in, abduct]",NaN,"[captive,hostage,lock-in,abduct]",NaN
52,Attack,"[Atack, Attack, Throw, threw, pelt, torch, cho...",NaN,"[atack,attack,throw,threw,pelt,torch,choas,los...",NaN
53,Disrupt,"[Tyre, Block, tire, burn, Fire, Bricks, Stones...",NaN,"[tyre,block,tire,burn,fire,bricks,stones,rubbi...",NaN
54,M&M,[March],[Memorandum],[march],[memorandum]
55,Vandalism,"[damaged, vandalised, break, broke]",NaN,"[damaged,vandalised,break,broke]",NaN


#### Actors

In [ ]:
actors = vocab.loc[vocab['Label'] == 'Actors']
actors = actors.drop('Label', axis = 1)
actors.to_excel('DATA/Vocabularies/actors.xlsx')
actors.shape

(4, 5)

In [ ]:
actors.head()

,Concept,Phrases,Conditional_Phrases,cleaned_phrases,cleaned_conditional_phrases
66,Union,"[COSATU, FOSATU, NEHAWU, SATAWU, Allied, AMCU,...",NaN,"[cosatu,fosatu,nehawu,satawu,allied,amcu,nuhhr...",NaN
67,Political Party,"[ANC, Congress, DA, EFF, IFP, NFP, Cope, UDM, ...",NaN,"[anc,congress,da,eff,ifp,nfp,cope,udm,sacp,acdp]",NaN
68,Civic org,"[association, residents, concerned, ratepayers...",NaN,"[association,residents,concerned,ratepayers,ta...",NaN
69,Church,"[church, congregation, believers, temple, mosq...",NaN,"[church,congregation,believers,temple,mosque,s...",NaN


#### Locations

In [ ]:
locations = vocab.loc[vocab['Label'] == 'Location']
locations = locations.drop('Label', axis = 1)
locations.to_excel('DATA/Vocabularies/locations.xlsx')
locations.shape

(7, 5)

In [ ]:
locations.head()

,Concept,Phrases,Conditional_Phrases,cleaned_phrases,cleaned_conditional_phrases
71,Informal area,"[Shacks, sharks, informal, hostel, squatter, c...",NaN,"[shacks,sharks,informal,hostel,squatter,camp,t...",NaN
72,Court,"[Court, hearing]",NaN,"[court,hearing]",NaN
73,Business premises,"[company, business, gate, premises, factory]",NaN,"[company,business,gate,premises,factory]",NaN
74,Stadium,"[Stadium, sports field]",NaN,"[stadium,sports, field]",NaN
75,Church,"[church, congregation, believers, temple, musl...",NaN,"[church,congregation,believers,temple,muslim,j...",NaN


#### Weapons

In [ ]:
weapons = vocab.loc[vocab['Label'] == 'Weapons']
weapons = weapons.drop('Label', axis = 1)
weapons.to_excel('DATA/Vocabularies/weapons.xlsx')
weapons.shape

(3, 5)

In [ ]:
weapons.head()

,Concept,Phrases,Conditional_Phrases,cleaned_phrases,cleaned_conditional_phrases
79,Crowd weapons,"[Weapon, Knife, Traditional weapon, knobkierie...",NaN,"[weapon,knife,traditional, weapon,knobkierie,k...",NaN
80,Crowd projectiles,"[Throw, Threw, Attack, attacked ]","[Stone, rock, brick, brike ]","[throw,threw,attack,attacked]","[stone,rock,brick,brike]"
81,Police weapons,"[Gun, cannon, canon, shotgun, s/gun, rubber, s...",NaN,"[gun,cannon,canon,shotgun,s/gun,rubber,stungre...",NaN


#### Eventuality

In [ ]:
eventuality = vocab.loc[vocab['Label'] == 'Eventuality']
eventuality = eventuality.drop('Label', axis = 1)
eventuality.to_excel('DATA/Vocabularies/eventuality.xlsx')
eventuality.shape

(2, 5)

In [ ]:
eventuality.head()

,Concept,Phrases,Conditional_Phrases,cleaned_phrases,cleaned_conditional_phrases
83,Police attack,"[Police, SAPS, officer]","[beat, raid, injury, rubber, stungrenade, stun...","[police,saps,officer]","[beat,raid,injury,rubber,stungrenade,stuntgren..."
84,Vigilantism,"[Beat, beating, mob justice, necklace, necklac...",NaN,"[beat,beating,mob, justice,necklace,necklacing...",NaN


#### Curiosity

In [ ]:
curiosity = vocab.loc[vocab['Label'] == 'Curiosity']
curiosity = curiosity.drop('Label', axis = 1)
curiosity.to_excel('DATA/Vocabularies/curiosity.xlsx')
curiosity.shape

(2, 5)

In [ ]:
curiosity.head()

,Concept,Phrases,Conditional_Phrases,cleaned_phrases,cleaned_conditional_phrases
86,Movements,"[Rebel, rebellion, uprising, movement]",NaN,"[rebel,rebellion,uprising,movement]",NaN
87,Special Keywords,"[Lunch, scab, scabs, swearing, loot, looting, ...",NaN,"[lunch,scab,scabs,swearing,loot,looting,sabc,p...",NaN


#### Non-protest

In [ ]:
non_protest = vocab.loc[vocab['Label'] == 'Non-protest']
non_protest = non_protest.drop('Label', axis = 1)
non_protest.to_excel('DATA/Vocabularies/non_protest.xlsx')
non_protest.shape

(11, 5)

In [ ]:
non_protest.head()

,Concept,Phrases,Conditional_Phrases,cleaned_phrases,cleaned_conditional_phrases
89,Football matches,"[Football, soccer, PSL, score, Bafana]",NaN,"[football,soccer,psl,score,bafana]",NaN
90,Other sport,"[Cricket, rugby, game, athletics, race, marath...",NaN,"[cricket,rugby,game,athletics,race,marathon,pr...",NaN
91,Tribal court meetings,"[Tribal court, traditional court, imbizo]",NaN,"[tribal, court,traditional, court,imbizo]",NaN
92,Funerals,"[Memorial, funeral, burial, after tears, mourn...",NaN,"[memorial,funeral,burial,after, tears,mourners...",NaN
93,Church,"[Church service, ZCC, Shembe, prayer, dedicati...",NaN,"[church, service,zcc,shembe,prayer,dedication]",NaN
